In [17]:
import os
import re
import numpy as np
import cv2
from collections import defaultdict
from random import randrange
from ultralytics import YOLO
from ultralytics.utils.torch_utils import select_device
from ocsort import OCSort

In [1]:
# Utility functions
def create_output_directory(output_path):
    try:
        os.makedirs(output_path, exist_ok=True)
        print(f"Directory created at: {output_path}")
    except OSError as e:
        print(f"Error creating directory: {e}")

def list_files(directory):
    try:
        return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    except Exception as e:
        print(f"Error listing files in directory {directory}: {e}")
        return []

def get_sorted_file_paths(directory):
    files = list_files(directory)
    return [os.path.join(directory, f"frame_{i:05d}.png") for i in range(1, len(files) + 1)]

def write_mot_format(data, filename):
    try:
        with open(filename, "w") as f:
            for entry in data:
                line = "{},{},{},{},{},{},{},{},{},{}\n".format(*entry)
                f.write(line)
        print(f"MOT data written to {filename}")
    except Exception as e:
        print(f"Error writing to file {filename}: {e}")

# Core functions
def initialize_model(model_path):
    model = YOLO(model_path, 'detect')
    device = select_device('cpu')
    model.to(device)
    return model

def process_frame(frame, model, tracker, track_history, frame_id, colors):
    prediction = model.predict(frame)
    prediction_results = prediction[0].boxes.cpu().numpy()
    if len(prediction_results) == 0:
        pred_concatenate = np.empty((0, 5))
    else:
        pred_conf = prediction_results.conf.reshape(prediction_results.conf.shape[0], 1)
        pred_concatenate = np.concatenate((prediction_results.xyxy, pred_conf), axis=1)

    tracks = tracker.update(pred_concatenate, img_info=[1088, 1920], img_size=[1088, 1920])

    if len(tracks) == 0:
        return

    print(tracks)
    bbox_list = tracks[:, 0:4] 
    track_id_list = tracks[:, 4]

    for track_id, bbox in zip(track_id_list, bbox_list):
        if track_id not in colors:
            colors[track_id] = (randrange(255), randrange(255), randrange(255))

        x0, y0, x1, y1 = map(int, bbox)
        track_history.append((frame_id, track_id, x0, y0, x1 - x0, y1 - y0, -1, -1, -1, -1))

def process_video(dir_images, model, tracker, output_file):
    files_sorted = get_sorted_file_paths(dir_images)
    track_history = []
    colors = {}

    for file in files_sorted:
        match = re.search(r'frame_\d+\.png', file)
        if not match:
            continue

        frame_id = int(match.group(0).split("_")[1].split(".")[0])
        frame = cv2.imread(file)
        process_frame(frame, model, tracker, track_history, frame_id, colors)

    write_mot_format(track_history, output_file)

In [2]:
# Global constants
ALGORITHM = "ocsort"
MODEL_NAME = "yolo11n_DJI_0008_V@fine-tuning" 
#"yolo11x_DJI_0008_V_and_0010_V_2@fine-tuning" 
# yolo11n_DJI_0010_V_2@fine-tuning 
# yolo11n_DJI_0008_V@fine-tuning
OUTPUT_PATH_MOT_FILES = f'../../output/mot_detections/{ALGORITHM}'
FRAME_RATE = 25
FRAME_WIDTH = 1920
FRAME_HEIGHT = 1088

create_output_directory(OUTPUT_PATH_MOT_FILES)

NameError: name 'os' is not defined

In [39]:
name_video = "DJI_20240308110013_0004_V_3"
# DJI_20240308110228_0006_V_2 DJI_20240308111117_0010_V_1 DJI_20240308110115_0005_V 
# DJI_20240308110013_0004_V_1 DJI_20240308110013_0004_V_3
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = OCSort(det_thresh=0.20)

process_video(dir_images, model, tracker, output_file)

Directory created at: ../../output/mot_detections/ocsort\yolo11n_DJI_0008_V@fine-tuning@DJI_20240308110013_0004_V_3


Ultralytics 8.3.11  Python-3.8.18 torch-2.1.0 CPU (12th Gen Intel Core(TM) i5-1235U)

0: 384x640 (no detections), 886.7ms
Speed: 231.4ms preprocess, 886.7ms inference, 85.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 195.4ms
Speed: 6.0ms preprocess, 195.4ms inference, 16.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 pineapples, 177.7ms
Speed: 4.3ms preprocess, 177.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 pineapples, 267.5ms
Speed: 8.4ms preprocess, 267.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 pineapples, 202.5ms
Speed: 3.5ms preprocess, 202.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 pineapples, 178.9ms
Speed: 4.8ms preprocess, 178.9ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 pineapples, 180.7ms
Speed: 3.0ms preprocess, 180.7ms inference, 3.0ms postprocess per image at shape (

In [40]:
name_video = "DJI_20240308110228_0006_V_2"
# DJI_20240308110228_0006_V_2 DJI_20240308111117_0010_V_1 DJI_20240308110115_0005_V 
# DJI_20240308110013_0004_V_1 DJI_20240308110013_0004_V_3
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = OCSort(det_thresh=0.20)

process_video(dir_images, model, tracker, output_file)

Directory created at: ../../output/mot_detections/ocsort\yolo11n_DJI_0008_V@fine-tuning@DJI_20240308110228_0006_V_2
Ultralytics 8.3.11  Python-3.8.18 torch-2.1.0 CPU (12th Gen Intel Core(TM) i5-1235U)

0: 384x640 (no detections), 167.9ms
Speed: 0.0ms preprocess, 167.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 138.3ms
Speed: 0.0ms preprocess, 138.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 218.3ms
Speed: 0.0ms preprocess, 218.3ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 158.1ms
Speed: 3.9ms preprocess, 158.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pineapples, 154.8ms
Speed: 5.4ms preprocess, 154.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 144.9ms
Speed: 0.0ms preprocess, 144.9ms inference, 0.0ms postprocess per image at shape (1, 3, 38

In [41]:
name_video = "DJI_20240308111117_0010_V_1"
# DJI_20240308110228_0006_V_2 DJI_20240308111117_0010_V_1 DJI_20240308110115_0005_V 
# DJI_20240308110013_0004_V_1 DJI_20240308110013_0004_V_3
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = OCSort(det_thresh=0.20)

process_video(dir_images, model, tracker, output_file)

Directory created at: ../../output/mot_detections/ocsort\yolo11n_DJI_0008_V@fine-tuning@DJI_20240308111117_0010_V_1
Ultralytics 8.3.11  Python-3.8.18 torch-2.1.0 CPU (12th Gen Intel Core(TM) i5-1235U)

0: 384x640 9 pineapples, 154.5ms
Speed: 0.0ms preprocess, 154.5ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 pineapples, 159.2ms
Speed: 0.0ms preprocess, 159.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 pineapples, 178.7ms
Speed: 3.0ms preprocess, 178.7ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 9 pineapples, 154.6ms
Speed: 3.0ms preprocess, 154.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 pineapples, 132.3ms
Speed: 1.9ms preprocess, 132.3ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 pineapples, 134.6ms
Speed: 2.0ms preprocess, 134.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 3

In [42]:
name_video = "DJI_20240308110115_0005_V"
# DJI_20240308110228_0006_V_2 DJI_20240308111117_0010_V_1 DJI_20240308110115_0005_V 
# DJI_20240308110013_0004_V_1 DJI_20240308110013_0004_V_3
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = OCSort(det_thresh=0.20)

process_video(dir_images, model, tracker, output_file)

Directory created at: ../../output/mot_detections/ocsort\yolo11n_DJI_0008_V@fine-tuning@DJI_20240308110115_0005_V
Ultralytics 8.3.11  Python-3.8.18 torch-2.1.0 CPU (12th Gen Intel Core(TM) i5-1235U)

0: 384x640 1 pineapple, 118.3ms
Speed: 11.6ms preprocess, 118.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pineapples, 151.4ms
Speed: 2.0ms preprocess, 151.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 pineapples, 138.3ms
Speed: 3.0ms preprocess, 138.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 pineapples, 177.5ms
Speed: 4.4ms preprocess, 177.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 153.0ms
Speed: 0.0ms preprocess, 153.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 pineapple, 129.6ms
Speed: 5.6ms preprocess, 129.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640

In [43]:
name_video = "DJI_20240308110013_0004_V_1"
# DJI_20240308110228_0006_V_2 DJI_20240308111117_0010_V_1 DJI_20240308110115_0005_V 
# DJI_20240308110013_0004_V_1 DJI_20240308110013_0004_V_3
model_path = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/weights/{MODEL_NAME}/best.pt"
dir_images = f"C:/Users/dnnxl/Documents/GitHub/drone-sort/dataset/{name_video}/valid/images"
output_dir = os.path.join(OUTPUT_PATH_MOT_FILES, f"{MODEL_NAME}@{name_video}")
output_file = os.path.join(output_dir, f"{name_video}.txt")

create_output_directory(output_dir)

model = initialize_model(model_path)
tracker = OCSort(det_thresh=0.20)

process_video(dir_images, model, tracker, output_file)

Directory created at: ../../output/mot_detections/ocsort\yolo11n_DJI_0008_V@fine-tuning@DJI_20240308110013_0004_V_1
Ultralytics 8.3.11  Python-3.8.18 torch-2.1.0 CPU (12th Gen Intel Core(TM) i5-1235U)

0: 384x640 8 pineapples, 179.6ms
Speed: 4.1ms preprocess, 179.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 pineapples, 199.7ms
Speed: 2.1ms preprocess, 199.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 pineapples, 174.1ms
Speed: 5.0ms preprocess, 174.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 pineapples, 137.6ms
Speed: 11.8ms preprocess, 137.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 pineapples, 114.3ms
Speed: 2.4ms preprocess, 114.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 pineapples, 127.9ms
Speed: 12.0ms preprocess, 127.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 3

#### Testing purposes

In [ ]:
files_sorted = get_sorted_file_paths(dir_images)
prediction = model.predict(files_sorted[0])
prediction_results = prediction[0].boxes.cpu().numpy()
pred_conf = prediction_results.conf.reshape(1, 1)
result = np.concatenate((prediction_results.xyxy, pred_conf), axis=1)
tracks = tracker.update(result, img_info=[1088, 1920], img_size=[1088, 1920])

bbox_list = tracks[:, 0:4] 
track_id_list = tracks[:, 4]